# Módulo de preprocesamiento 
Ammi Beltrán & Fernanda Borja

## 1

In [1]:
import os
import mne
import glob
#
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import copy

In [2]:
if not os.path.exists("data"):
    os.makedirs("data")
    print("Data directory created :D")

In [3]:
EDFDIR = "D:\\OneDrive\\OneDrive - Universidad de Chile\\Semestre X\\Inteligencia\\Proyecto\\dataset\\tuh_eeg"

In [9]:
# EDFDIR = "c:\\Users\\TheSy\\Desktop\\tuh_eeg"

In [4]:
files = glob.glob(EDFDIR + '/**/*.edf', recursive=True)

In [97]:
# Usamos MNE
data = mne.io.read_raw_edf(files[0])
raw_data = data.get_data()
info = data.info
channels = data.ch_names

Extracting EDF parameters from D:\OneDrive\OneDrive - Universidad de Chile\Semestre X\Inteligencia\Proyecto\dataset\tuh_eeg\aaaaaaaa\s001_2015_12_30\01_tcp_ar\aaaaaaaa_s001_t000.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


In [98]:
int(data.times[-1])

1264

In [104]:
data.ch_names

['EEG FP1-REF',
 'EEG FP2-REF',
 'EEG F3-REF',
 'EEG F4-REF',
 'EEG C3-REF',
 'EEG C4-REF',
 'EEG P3-REF',
 'EEG P4-REF',
 'EEG O1-REF',
 'EEG O2-REF',
 'EEG F7-REF',
 'EEG F8-REF',
 'EEG T3-REF',
 'EEG T4-REF',
 'EEG T5-REF',
 'EEG T6-REF',
 'EEG T1-REF',
 'EEG T2-REF',
 'EEG FZ-REF',
 'EEG CZ-REF',
 'EEG PZ-REF',
 'EEG EKG1-REF',
 'EEG LOC-REF',
 'EEG ROC-REF',
 'EMG-REF',
 'EEG A1-REF',
 'EEG A2-REF',
 'RESP ABDOMEN-REF',
 'IBI',
 'BURSTS',
 'SUPPR']

In [99]:
data.info.ch_names

['EEG FP1-REF',
 'EEG FP2-REF',
 'EEG F3-REF',
 'EEG F4-REF',
 'EEG C3-REF',
 'EEG C4-REF',
 'EEG P3-REF',
 'EEG P4-REF',
 'EEG O1-REF',
 'EEG O2-REF',
 'EEG F7-REF',
 'EEG F8-REF',
 'EEG T3-REF',
 'EEG T4-REF',
 'EEG T5-REF',
 'EEG T6-REF',
 'EEG T1-REF',
 'EEG T2-REF',
 'EEG FZ-REF',
 'EEG CZ-REF',
 'EEG PZ-REF',
 'EEG EKG1-REF',
 'EEG LOC-REF',
 'EEG ROC-REF',
 'EMG-REF',
 'EEG A1-REF',
 'EEG A2-REF',
 'RESP ABDOMEN-REF',
 'IBI',
 'BURSTS',
 'SUPPR']

In [48]:
len(files)

454

In [106]:
'''
CHANNELS = ['EEG FP1-REF','EEG FP2-REF',
            'EEG F7-REF','EEG F8-REF','EEG F4-REF','EEG F3-REF','EEG FZ-REF',
            'EEG A1-REF','EEG A2-REF','EEG O1-REF','EEG O2-REF',
            'EEG T6-REF','EEG T5-REF','EEG T4-REF','EEG T3-REF',
            'EEG C4-REF','EEG C3-REF','EEG CZ-REF',
            'EEG P4-REF','EEG P3-REF','EEG PZ-REF',
            ]
CHANNELS_LE = [
            'EEG FP1-LE',
            'EEG FP2-LE',
            'EEG F7-LE',
            'EEG F8-LE',
            'EEG F4-LE',
            'EEG F3-LE',
            'EEG FZ-LE',
            'EEG A1-LE',
            'EEG A2-LE',
            'EEG O1-LE',
            'EEG O2-LE',
            'EEG T6-LE',
            'EEG T5-LE',
            'EEG T4-LE',
            'EEG T3-LE',
            'EEG C4-LE',
            'EEG C3-LE',
            'EEG CZ-LE',
            'EEG P4-LE',
            'EEG P3-LE',
            'EEG PZ-LE',
            ]
'''
CHANNELS = [
            'FP1',
            'FP2',
            'F7',
            'F8',
            'F4',
            'F3',
            'FZ',
            'A1',
            'A2',
            'O1',
            'O2',
            'T6',
            'T5',
            'T4',
            'T3',
            'C4',
            'C3',
            'CZ',
            'P4',
            'P3',
            'PZ',
            ]
def preprocessing(path):
    ''' 
    Pipeline de procesamiento c:
    - Canales c:
    - Filtrado c:
    - Downsample c:
    - Crop temporal (primer minuto y a los 11 minutos) c: 
    - Segmentación c:
    - Normalización c:
    '''
    files = glob.glob(path + '/**/*.edf', recursive = True)
    # windows = np.empty() EVALUAR
    windows = []
    for i in range(0, len(files)):
        data = mne.io.read_raw_edf(files[i], preload=True)
        # raw_data = data.get_data()
        # info = data.info
        # channels = data.ch_names
        # Pipeline
        # 
        mapping = rename_channels(data)
        data.rename_channels(data.info, mapping)
        # try:
        channel_data = channel_select(data, CHANNELS)
        ch = CHANNELS
        # except:
        #     channel_data = channel_select(data, CHANNELS_LE)
        #     ch = CHANNELS_LE
        if(int(data.times[-1]) < 80):
            continue
        filtered = eeg_filter(channel_data)
        trimmed_data = temporal_crop(filtered)
        epochs = get_epochs(trimmed_data, ch)
        down_data = downsample(epochs)
        norm_data = normalization(down_data)
        #
        windows.append(norm_data)
    return windows


In [108]:
# Channels we want
CHANNELS = [
            'FP1',
            'FP2',
            'F7',
            'F8',
            'F4',
            'F3',
            'FZ',
            'A1',
            'A2',
            'O1',
            'O2',
            'T6',
            'T5',
            'T4',
            'T3',
            'C4',
            'C3',
            'CZ',
            'P4',
            'P3',
            'PZ',
            ]
def rename_channels(data):
    # info = data.info
    original = data.ch_names
    print(original)
    new = copy.copy(original)
    new = [original[i] if (new[i] not in original[i]) else new[i] for i in range(0, len(new))]
    res = {original[i]: new[i] for i in range(len(new))}
    return res
    

In [109]:
preprocessing(EDFDIR)

Extracting EDF parameters from D:\OneDrive\OneDrive - Universidad de Chile\Semestre X\Inteligencia\Proyecto\dataset\tuh_eeg\aaaaaaaa\s001_2015_12_30\01_tcp_ar\aaaaaaaa_s001_t000.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 323839  =      0.000 ...  1264.996 secs...
['EEG FP1-REF', 'EEG FP2-REF', 'EEG F3-REF', 'EEG F4-REF', 'EEG C3-REF', 'EEG C4-REF', 'EEG P3-REF', 'EEG P4-REF', 'EEG O1-REF', 'EEG O2-REF', 'EEG F7-REF', 'EEG F8-REF', 'EEG T3-REF', 'EEG T4-REF', 'EEG T5-REF', 'EEG T6-REF', 'EEG T1-REF', 'EEG T2-REF', 'EEG FZ-REF', 'EEG CZ-REF', 'EEG PZ-REF', 'EEG EKG1-REF', 'EEG LOC-REF', 'EEG ROC-REF', 'EMG-REF', 'EEG A1-REF', 'EEG A2-REF', 'RESP ABDOMEN-REF', 'IBI', 'BURSTS', 'SUPPR']


ValueError: Invalid channel name(s) {'ctf_head_t', 'custom_ref_applied', 'kit_system_id', 'proj_id', 'experimenter', 'nchan', 'line_freq', 'hpi_subsystem', 'acq_stim', 'chs', 'helium_info', 'hpi_results', 'sfreq', 'utc_offset', 'proj_name', 'meas_date', 'hpi_meas', 'bads', 'acq_pars', 'device_info', 'xplotter_layout', 'comps', 'dev_ctf_t', 'highpass', 'subject_info', 'meas_id', 'gantry_angle', 'dev_head_t', 'ch_names', 'projs', 'lowpass', 'dig', 'proc_history', 'events', 'description', 'file_id'} are not present in info

In [7]:
def channel_select(data, channels):
    '''
    Selects channels from array 
    '''
    extracted = data.pick(channels)
    return extracted

In [53]:
def eeg_filter(data, lfreq = 0.3, hfreq= 80):
    '''
    
    '''
    data_copy = copy.copy(data)
    filtered = data_copy.filter(l_freq = lfreq,
                                h_freq = hfreq,
                                method = "iir"
                                )
    return filtered

In [75]:
def temporal_crop(data, tin = 60, tfin = 12*60):
    ''' 
    Cut the channels from the second "tin" to "tfin"
    '''
    data_copy = copy.copy(data)
    croped = data_copy.crop(tmin = tin, tmax = min(tfin, int(data.times[-1])))
    return croped

In [13]:
# def downsample(data, factor):
#     ''' 
#     Downsamples the data given by a factor
#     En nuestro caso, down corresponde a (frecuencia que queremos)/(frecuencia actual)
#     '''
#     data_copy = copy.copy(data)
#     down = mne.filter.resample(data_copy, down = factor, npad = "auto")
#     return down


In [55]:
def downsample(epoch, freq = 200):
    ''' 
    Downsamples the data given by a factor
    En nuestro caso, down corresponde a (frecuencia que queremos)/(frecuencia actual)
    '''
    down = epoch.resample(freq, npad = "auto")
    return down

In [79]:
# eventos
def get_epochs(data, channels, window = 20.0):
    ''' 
    window es la ventana de tiempo
    '''
    data_copy = copy.copy(data)
    # Create events
    events = mne.make_fixed_length_events(data_copy, duration = window, first_samp = True)
    # Divide accordingly
    picks = channels
    epochs = mne.Epochs(raw = data_copy, events = events, picks = picks, preload = True,
                        tmin = 0., tmax=window, baseline = None)
    return epochs


In [57]:
def normalization(epochs):
    obj = mne.decoding.Scaler(info = epochs.info, scalings='mean')
    values = obj.fit_transform(epochs.get_data())
    return values

In [23]:
a = get_epochs(data)

Not setting metadata
63 matching events found
No baseline correction applied
0 projection items activated
Loading data for 63 events and 5121 original time points ...
0 bad epochs dropped


In [24]:
a

Number of events,63
Events,1: 63
Time range,0.000 – 20.000 s
Baseline,off


In [28]:
b = downsample(a, 200)

In [32]:
b.info

<Info | 8 non-empty values
 bads: []
 ch_names: EEG FP1-REF, EEG FP2-REF, EEG F7-REF, EEG F8-REF, EEG F4-REF, ...
 chs: 21 EEG
 custom_ref_applied: False
 highpass: 0.0 Hz
 lowpass: 100.0 Hz
 meas_date: 2015-12-30 14:44:21 UTC
 nchan: 21
 projs: []
 sfreq: 200.0 Hz
 subject_info: 3 items (dict)
>

In [10]:
info

<Info | 8 non-empty values
 bads: []
 ch_names: EEG FP1-REF, EEG FP2-REF, EEG F3-REF, EEG F4-REF, EEG C3-REF, ...
 chs: 31 EEG
 custom_ref_applied: False
 highpass: 0.0 Hz
 lowpass: 128.0 Hz
 meas_date: 2015-12-30 14:44:21 UTC
 nchan: 31
 projs: []
 sfreq: 256.0 Hz
 subject_info: 3 items (dict)
>

In [11]:
# info["sfreq"]
info["bads"]

[]

In [12]:
# plt.plot(raw_data[0][:250*60])
# plt.show()